In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df=pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations=city_data_df[["Lat","Lng"]]

# Get the maximum temperature non negative.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations=city_data_df[["Lat","Lng"]]
humidity=city_data_df["Humidity"]

fig=gmaps.figure(center=(30.00,31.00), zoom_level=1.5)

heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=4 )

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Percent Cloudiness Heatmap
locations=city_data_df[["Lat","Lng"]]
cloudiness=city_data_df["Cloudiness"]

fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

heat_layer=gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Wind Speed Heatmap
locations=city_data_df[["Lat","Lng"]]
wind=city_data_df["Wind Speed"]

fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

heat_layer=gmaps.heatmap_layer(locations, weights=wind, dissipating=False, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <=max_temp)&
                                    (city_data_df["Max Temp"]>= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Rikitea,PF,2022-02-02 22:15:29,-23.1203,-134.9692,78.46,73,100,7.78
7,7,Avarua,CK,2022-02-02 22:16:34,-21.2078,-159.7750,82.45,78,75,18.41
13,13,Vaini,TO,2022-02-02 22:17:49,-21.2000,-175.2000,86.16,74,40,5.75
14,14,Mumford,GH,2022-02-02 22:17:50,5.2618,-0.7590,80.20,85,1,8.95
15,15,Georgetown,MY,2022-02-02 22:14:43,5.4112,100.3354,80.53,85,20,2.30


In [10]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Rikitea,PF,78.46,-23.1203,-134.9692,
7,Avarua,CK,82.45,-21.2078,-159.7750,
13,Vaini,TO,86.16,-21.2000,-175.2000,
14,Mumford,GH,80.20,5.2618,-0.7590,
15,Georgetown,MY,80.53,5.4112,100.3354,
17,Mahebourg,MU,79.25,-20.4081,57.7000,
18,Necochea,AR,84.02,-38.5473,-58.7368,
22,Hilo,US,80.33,19.7297,-155.0900,
39,Vao,NC,77.74,-22.6667,167.4833,
43,Butaritari,KI,81.73,3.0707,172.7902,


In [15]:
import json

In [22]:
# Set parameters to search for a hotel.
params={
    "radius":5000,
    "type":"lodge",
    "key":g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [23]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Rikitea,PF,78.46,-23.1203,-134.9692,Rikitea
7,Avarua,CK,82.45,-21.2078,-159.7750,Avarua District
13,Vaini,TO,86.16,-21.2000,-175.2000,Nukuhetulu
14,Mumford,GH,80.20,5.2618,-0.7590,Apam
15,Georgetown,MY,80.53,5.4112,100.3354,George Town
...,...,...,...,...,...,...
520,Bundaberg,AU,81.21,-24.8500,152.3500,Bundaberg
522,Acajutla,SV,84.79,13.5928,-89.8275,Acajutla
527,Aloleng,PH,75.42,16.1307,119.7824,Agno
529,Kalmunai,LK,75.18,7.4167,81.8167,Kalmunai


In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add a heatmap of temperature for the vacation spots.
locations=hotel_df[["Lat","Lng"]]
max_temp=hotel_df["Max Temp"]

fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                               max_intensity=300, point_radius=4)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))